# Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text

# Instância de conexão com o banco de dados

In [2]:
usuario = "root"
senha = "root"
url = "localhost"
porta = "3306"
banco = "ENEM_OLTP"

# Criar uma conexão com o banco de dados MySQL
engine = create_engine(f"mysql+pymysql://{usuario}:{senha}@{url}:{porta}/{banco}")

# Instância de conexão com o banco de dados DataMart

In [3]:
usuario = "root"
senha = "root"
url = "localhost"
porta = "3306"
banco = "ENEM_DM"

# Criar uma conexão com o banco de dados MySQL
engine_dm = create_engine(f"mysql+pymysql://{usuario}:{senha}@{url}:{porta}/{banco}")

# Mapeamento dos microdados

In [4]:
micro_dados = [
    "FAIXA_ETARIA",
    "SEXO",
    "ESTADO_CIVIL",
    "COR_RACA",
    "NACIONALIDADE",
    "ST_CONCLUSAO",
    "ANO_CONCLUIU",
    "ESCOLA",
    "ENSINO",
    "TREINEIRO",
    "PRESENCA_CN",
    "PRESENCA_CH",
    "PRESENCA_LC",
    "PRESENCA_MT",
    "LINGUA",
    "STATUS_REDACAO",
] + [f"RESP_QUESTAO_{i}" for i in range(1, 26)]

# Criação das tabelas dimensões no DataMart

In [5]:
for tabela in micro_dados:
    query = f"SELECT * FROM {tabela}"
    df = pd.read_sql(query, con=engine)

    colunas = df.columns.tolist()

    df[f"SK_{tabela}"] = np.arange(1, len(df) + 1)

    df = df[[f"SK_{tabela}"] + colunas]

    with engine_dm.connect() as connection:
        connection.execute(text(f"DROP TABLE IF EXISTS DIM_{tabela}"))
        df.to_sql(f"DIM_{tabela}", con=connection, index=False)
        connection.execute(
            text(
                f"""
                ALTER TABLE DIM_{tabela}
                ADD PRIMARY KEY (SK_{tabela})
                """
            )
        )

    print(f"Dimensão {tabela} criada com sucesso! Total de registros: {len(df)}")

Dimensão FAIXA_ETARIA criada com sucesso! Total de registros: 20
Dimensão SEXO criada com sucesso! Total de registros: 2
Dimensão ESTADO_CIVIL criada com sucesso! Total de registros: 5
Dimensão COR_RACA criada com sucesso! Total de registros: 7
Dimensão NACIONALIDADE criada com sucesso! Total de registros: 5
Dimensão ST_CONCLUSAO criada com sucesso! Total de registros: 4
Dimensão ANO_CONCLUIU criada com sucesso! Total de registros: 18
Dimensão ESCOLA criada com sucesso! Total de registros: 3
Dimensão ENSINO criada com sucesso! Total de registros: 2
Dimensão TREINEIRO criada com sucesso! Total de registros: 2
Dimensão PRESENCA_CN criada com sucesso! Total de registros: 3
Dimensão PRESENCA_CH criada com sucesso! Total de registros: 3
Dimensão PRESENCA_LC criada com sucesso! Total de registros: 3
Dimensão PRESENCA_MT criada com sucesso! Total de registros: 3
Dimensão LINGUA criada com sucesso! Total de registros: 2
Dimensão STATUS_REDACAO criada com sucesso! Total de registros: 8
Dimensão